# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     |Sara Hernandez Ochoa   |
| **Fecha**      15/09/2025|   |
| **Expediente**750733 |   |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Para dividir los datos en dos conjuntos, uno de entrenamiento y otro de prueba. Para que el modelo aprenda con una parte de los datos y luego se evalúe con información que no ha visto antes. Asi se verifica si el modelo generaliza bien y no solo memoriza.

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

Aunque usar todos los datos parece mejor, si no se reserva un conjunto para prueba no se puede saber si el modelo generaliza. Simula información nueva y estimar el error real. Por eso se combina entrenamiento con evaluación, o se usa validación cruzada para aprovechar mejor la muestra.

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [22]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import scipy.stats as st

In [36]:
data = pd.read_excel("Motor Trend Car Road Tests.xlsx")

In [37]:
data=data.drop(columns=['model'])

In [38]:
mse = []

for i in range(len(data)):
    # Separar test y train
    data_test = data.iloc[[i]]        
    data_train = data.drop(i)         
    
    X_train = data_train.drop(columns=['mpg'])
    y_train = data_train['mpg']

    X_test = data_test.drop(columns=['mpg'])
    y_test = data_test['mpg']

    # Entrenar LinearRegression
    lr = LinearRegression()
    reg = lr.fit(X_train, y_train)

    # Predecir y calcular MSE
    y_pred_test = reg.predict(X_test)
    error = mean_squared_error(y_test, y_pred_test)
    mse.append(error)

In [39]:
media = np.mean(mse)                    # MSE promedio
desv = np.std(mse, ddof=1)              # Desviación estándar muestral
n = len(mse)
alpha = 0.05
t_crit = st.t.ppf(1 - alpha/2, df=n-1)  # t crítico

# Margen de error e intervalo de confianza
margen_error = t_crit * (desv / np.sqrt(n))
IC = (media - margen_error, media + margen_error)

# Mostrar resultados
print("MSE promedio:", media)
print("Desviación estándar del MSE:", desv)
print("Intervalo de confianza 95%:", IC)

MSE promedio: 12.181558006901954
Desviación estándar del MSE: 17.340495610343044
Intervalo de confianza 95%: (5.929642648176542, 18.433473365627364)


Interpreta.

El MSE promedio es 12.18, lo que indica que, en promedio, las predicciones del modelo difieren de los valores reales en un error cuadrático de aproximadamente 12.

La desviación estándar del MSE es 17.34, lo que muestra que los errores individuales varían bastante entre las diferentes iteraciones de LOOCV; algunas predicciones son mucho mejores o peores que otras.

El intervalo de confianza del 95% va de 5.93 a 18.43, lo que significa que podemos estar razonablemente seguros de que el MSE “real” del modelo (si lo aplicáramos a datos similares) se encuentra dentro de ese rango.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [ ]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

In [41]:
# Ejercicio 2: K-Fold Cross Validation con California Housing

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import KFold

# Cargar dataset
housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)

X = housing.data
y = housing.target

# Definir K-Folds
kf = KFold(n_splits=10, shuffle=True, random_state=100)

mse_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    lr = LinearRegression()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_test)
    error = mean_squared_error(y_test, y_pred)
    mse_scores.append(error)

# Calcular estadísticas
media = np.mean(mse_scores)
desv = np.std(mse_scores, ddof=1)
n = len(mse_scores)
alpha = 0.05
t_crit = st.t.ppf(1 - alpha/2, df=n-1)

margen_error = t_crit * (desv / np.sqrt(n))
IC = (media - margen_error, media + margen_error)

# Resultados
print("MSE por fold:", mse_scores)
print(f"MSE promedio: {media:.4f}")
print(f"Desviación estándar del MSE: {desv:.4f}")
print("Intervalo de confianza 95%:", IC)


Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
MSE por fold: [0.5117242819193435, 0.5073036455116077, 0.4841587761712365, 0.5241268400393478, 0.5330791923450088, 0.5217856372536162, 0.5217818134767094, 0.5606195714023077, 0.5178668381533028, 0.5863146794895464]
MSE promedio: 0.5269
Desviación estándar del MSE: 0.0285
Intervalo de confianza 95%: (0.5065194180131435, 0.547232837139262)


Interpreta.

El modelo de regresión lineal aplicado al dataset de California Housing mostró errores (MSE) en cada fold que van de 0.48 a 0.59, con un promedio de 0.527. Esto significa que, en general, el modelo predice con un error cercano a ese valor. La desviación estándar de 0.029 indica que los resultados son consistentes entre los diferentes folds, y el intervalo de confianza al 95% (0.507 – 0.547) confirma que el error esperado del modelo se encuentra con alta probabilidad dentro de ese rango.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3